In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Instructions:
1. download the datasets from our github repo
2. upload them into google drive in separate folders
3. cd to the parent directories of the uploaded folders

In [ ]:
%cd /content/drive/MyDrive/10623/project_eval/clip_report
%ls

/content/drive/MyDrive/10623/project_eval/clip_report
dataset/  output_baseline/


In [ ]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
# For offical pip
# !pip install pytorch>1.7.1 torchvision cudatoolkit
# !pip install ftfy regex tqdm datasets
# !pip install git+https://github.com/openai/CLIP.git

# For transformers pip
pip install transformers Pillow torch

ERROR: Could not find a version that satisfies the requirement cudatoolkit (from versions: none)
ERROR: No matching distribution found for cudatoolkit
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-tbklap9j
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-tbklap9j
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


# From CLIP official

In [ ]:
import torch
import clip
import PIL.Image


#modified from offical implementation of CLIP: https://github.com/openai/CLIP

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def calculate_clip(t1, t2, is_multimodal):
    t1_input = preprocess(PIL.Image.open(t1)).unsqueeze(0).to(device)

    if is_multimodal:
        t2_input = clip.tokenize(t2).to(device)
    else:
        t2_input = preprocess(PIL.Image.open(t2)).unsqueeze(0).to(device)

    with torch.no_grad():
        t1_features = model.encode_image(t1_input)

        if is_multimodal:
            t2_features = model.encode_text(t2_input)
        else:
            t2_features = model.encode_image(t2_input)

        t1_features /= t1_features.norm(dim=-1, keepdim=True)
        t2_features /= t2_features.norm(dim=-1, keepdim=True)
        similarity = (t1_features @ t2_features.T)

    return similarity



In [ ]:
%ls
%cd drive/MyDrive/genai_proj/instruct-pix2pix/clip

1.7.1  condacolab_install.log  drive/  sample_data/
/content/drive/MyDrive/genai_proj/instruct-pix2pix/clip


In [ ]:
%ls
%ls input_image
%ls output_image

input_image/  output_image/
1_a.jpeg  1_b.jpeg  2.jpeg  3.jpeg  4.jpeg
output1_a.jpeg  output1_b.jpeg  output2.jpeg  output3.jpeg  output4.jpeg


In [ ]:
from datasets import Dataset, Image

INPUT_PATH = "input_image/"
OUTPUT_PATH = "output_image/"

png_paths = ["1_a.jpeg", "1_b.jpeg", "2.jpeg", "3.jpeg", "4.jpeg"]
input_images = []
output_images = []

for i in range(0, len(png_paths)):
    input_images.append(f"{INPUT_PATH}{png_paths[i]}")
    output_images.append(f"{OUTPUT_PATH}output{png_paths[i]}")


for i in range(0, len(png_paths)):
    print(calculate_clip(input_images[i], output_images[i], False))

tensor([[0.8687]], device='cuda:0', dtype=torch.float16)
tensor([[0.6885]], device='cuda:0', dtype=torch.float16)
tensor([[0.7236]], device='cuda:0', dtype=torch.float16)
tensor([[0.8599]], device='cuda:0', dtype=torch.float16)
tensor([[0.6230]], device='cuda:0', dtype=torch.float16)


# Using diffusers pipelines for CLIP and DinoV2 scores
- Note, load specific pretrained CLIP on which stable diffusion is trained
- Also, modified from https://medium.com/aimonks/clip-vs-dinov2-in-image-similarity-6fa5aa7ed8c6

In [ ]:
!pip install transformers Pillow torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, CLIPModel
import torch.nn as nn
from transformers import AutoImageProcessor, AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [ ]:
clip_processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)

def get_clipI(i1_path, i2_path, device=device, processor=clip_processor, model=clip_model):
    #Extract features from image1
    with torch.no_grad():
        inputs1 = processor(images=Image.open(i1_path), return_tensors="pt").to(device)
        image_features1 = model.get_image_features(**inputs1)

    with torch.no_grad():
        inputs2 = processor(images=Image.open(i2_path), return_tensors="pt").to(device)
        image_features2 = model.get_image_features(**inputs2)

    #Compute their cosine similarity and convert it into a score between 0 and 1
    cos = nn.CosineSimilarity(dim=0)
    sim = cos(image_features1[0],image_features2[0]).item()
    sim = (sim+1)/2
    print('Similarity:', sim)
    return sim

In [ ]:
dino_processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
dino_model = AutoModel.from_pretrained('facebook/dinov2-base').to(device)

def get_dinoV2(i1_path, i2_path, device=device, processor=dino_processor, model=dino_model):
    with torch.no_grad():
        inputs1 = processor(images=Image.open(i1_path), return_tensors="pt").to(device)
        outputs1 = model(**inputs1)
        image_features1 = outputs1.last_hidden_state
        image_features1 = image_features1.mean(dim=1)

    with torch.no_grad():
        inputs2 = processor(images=Image.open(i2_path), return_tensors="pt").to(device)
        outputs2 = model(**inputs2)
        image_features2 = outputs2.last_hidden_state
        image_features2 = image_features2.mean(dim=1)

    cos = nn.CosineSimilarity(dim=0)
    sim = cos(image_features1[0],image_features2[0]).item()
    sim = (sim+1)/2
    print('Similarity:', sim)
    return sim

In [ ]:
%ls dataset

10.jpeg  15.jpeg  1_a.jpg  21.jpeg  26.jpeg  2_b.jpg  31.jpeg  37.jpeg  41.jpg  46.jpg  8.jpeg
11.jpeg  16.jpeg  1_b.jpg  22.jpeg  27.jpeg  2_c.jpg  33.jpeg  38.jpg   42.jpg  4.jpeg  9.jpeg
12.jpeg  17.jpeg  1_c.jpg  23.jpeg  28.jpeg  2_d.jpg  34.jpeg  39.jpg   43.jpg  5.jpeg
13.jpeg  18.jpeg  1.jpeg   24.jpeg  29.jpeg  2.jpeg   35.jpeg  3.jpeg   44.jpg  6.jpeg
14.jpeg  19.jpeg  20.jpeg  25.jpeg  2_a.jpg  30.jpeg  36.jpeg  40.jpg   45.jpg  7.jpeg


In [ ]:
%ls output_baseline

12.png  1_a.png  21.png  33.png  3.png   46.png  5.png  8.png
15.png  1_b.png  2.png   36.png  41.png  4.png   7.png


In [ ]:
%ls imgs_edited

1_a.jpg  1_b.jpg  21.jpeg  2.jpeg  36.jpeg  3.jpeg  41.jpg  46.jpg  7.jpeg  prompts.json


In [ ]:
INPUT_PATH = "dataset/"
OUTPUT_PATH = "output_baseline/"

in_paths = ["1_a.jpg",  "1_b.jpg",  "21.jpeg",  "2.jpeg",  "36.jpeg",  "3.jpeg",  "41.jpg",  "46.jpg",  "7.jpeg"]
out_paths = ["1_a.png",  "1_b.png",  "21.png",  "2.png",  "36.png",  "3.png",  "41.png",  "46.png",  "7.png"]
input_images = []
output_images = []

for i in range(0, len(in_paths)):
    input_images.append(f"{INPUT_PATH}{in_paths[i]}")
    output_images.append(f"{OUTPUT_PATH}{out_paths[i]}")


clip_total = 0
dino_total = 0
for i in range(0, len(in_paths)):
    clip_total += get_clipI(input_images[i], output_images[i])
    dino_total += get_dinoV2(input_images[i], output_images[i])

print(clip_total/len(in_paths))
print(dino_total/len(in_paths))

Similarity: 0.9463466703891754
Similarity: 0.9937996566295624
Similarity: 0.8090856969356537
Similarity: 0.6621357798576355
Similarity: 0.9793371856212616
Similarity: 0.9882122874259949
Similarity: 0.9554187655448914
Similarity: 0.9804982841014862
Similarity: 0.8865485787391663
Similarity: 0.960048258304596
Similarity: 0.8895609378814697
Similarity: 0.950719952583313
Similarity: 0.8307322263717651
Similarity: 0.7203241735696793
Similarity: 0.7651555240154266
Similarity: 0.8155746459960938
Similarity: 0.9890547692775726
Similarity: 0.9931381046772003
0.8945822616418203
0.8960501270161735


In [ ]:
INPUT_PATH = "dataset/"
OUTPUT_PATH = "imgs_edited/"

in_paths = ["1_a.jpg",  "1_b.jpg",  "21.jpeg",  "2.jpeg",  "36.jpeg",  "3.jpeg",  "41.jpg",  "46.jpg",  "7.jpeg"]
out_paths = ["1_a.jpg",  "1_b.jpg",  "21.jpeg",  "2.jpeg",  "36.jpeg",  "3.jpeg",  "41.jpg",  "46.jpg",  "7.jpeg"]
input_images = []
output_images = []

for i in range(0, len(in_paths)):
    input_images.append(f"{INPUT_PATH}{in_paths[i]}")
    output_images.append(f"{OUTPUT_PATH}{out_paths[i]}")

clip_total = 0
dino_total = 0
for i in range(0, len(in_paths)):
    clip_total += get_clipI(input_images[i], output_images[i])
    dino_total += get_dinoV2(input_images[i], output_images[i])

print(clip_total/len(in_paths))
print(dino_total/len(in_paths))

Similarity: 0.9093898236751556
Similarity: 0.9690873026847839
Similarity: 0.9542901515960693
Similarity: 0.9268201589584351
Similarity: 0.9300181865692139
Similarity: 0.9161568880081177
Similarity: 0.9189943969249725
Similarity: 0.9323628544807434
Similarity: 0.8749442100524902
Similarity: 0.8393747806549072
Similarity: 0.9176546633243561
Similarity: 0.9237892031669617
Similarity: 0.8879525661468506
Similarity: 0.7527952492237091
Similarity: 0.9682963490486145
Similarity: 0.9486328959465027
Similarity: 0.8996129035949707
Similarity: 0.9351735711097717
0.9179059167702993
0.9049103226926591
